In [1]:
import sys
sys.path.insert(0, '/Users/kylesargent/Dropbox/evlab/modeling 2/')
from dat import Dat
data = Dat()
data.add_dict('twitter')
data.make_gen_dict()
X,y = data.getXy()

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def jd(v,w): 
    ci  = np.sum(v)
    cj  = np.sum(w)
    cij = np.sum((v + w)//2)
    if cij == 0:
        return(1)
    else:
        return(1 - (cij)/(ci + cj - cij))
    
def cs(v,w):
    return(cosine_similarity([v], [w])[0][0])
    
from sklearn.cluster import SpectralClustering

n_frames = y.shape[1]
frame_dist_matrix = np.zeros((n_frames, n_frames))
for i in range(0, n_frames):
    for j in range(i, n_frames):
        d = jd(y[:,i],y[:,j])
        frame_dist_matrix[i][j] = d
        frame_dist_matrix[j][i] = d
        
n_verbs = y.shape[0]
verb_syntax_dist_matrix = np.zeros((n_verbs, n_verbs))
for i in range(0, n_verbs):
    for j in range(i, n_verbs):
        d = jd(y[i],y[j])
        verb_syntax_dist_matrix[i][j] = d
        verb_syntax_dist_matrix[j][i] = d
        
verb_cossim_dist_matrix = np.zeros((n_verbs, n_verbs))
for i in range(0, n_verbs):
    for j in range(i, n_verbs):
        d = cs(X[i],X[j])
        verb_cossim_dist_matrix[i][j] = d
        verb_cossim_dist_matrix[j][i] = d

KeyboardInterrupt: 

In [4]:
import scipy.io

scipy.io.savemat('fsd.mat', mdict={'frame_syntax_distance': frame_dist_matrix})
scipy.io.savemat('vsd.mat', mdict={'verb_syntax_distance': verb_syntax_dist_matrix})
scipy.io.savemat('vcd.mat', mdict={'verb_cossim_distance': verb_cossim_dist_matrix})

In [12]:
frame_cluster_labels = np.array(scipy.io.loadmat("cluster_labels.mat")["fcl"]).T[0]
verb_syntax_cluster_labels = np.array(scipy.io.loadmat("cluster_labels.mat")["vscl"]).T[0]
verb_cossim_cluster_labels = np.array(scipy.io.loadmat("cluster_labels.mat")["vccl"]).T[0]

from sklearn.metrics import adjusted_mutual_info_score as ami

print(max(verb_syntax_cluster_labels))    
print(max(verb_cossim_cluster_labels))
#np.mean(ami_list)

34
2


In [ ]:
# Determing list of diagnostic frames, 
        

        
aff = np.exp(- frame_dist_matrix ** 2 / (2. * 1 ** 2)) #Gaussian heat kernel

n_diagnostic_frames = 50

sc_frames = SpectralClustering(n_clusters = n_diagnostic_frames, affinity = "precomputed")
frame_cluster_predictions = sc_frames.fit_predict(aff)
frame_cluster_pairs = list(enumerate(frame_cluster_predictions))

In [ ]:
from sklearn.cluster import KMeans
glove_verbs = list(map(lambda v : (v[0]), list(gen_dict.items())))
glove_vecs = np.array(list(map(lambda v : (v[1][1]), list(gen_dict.items()))))

n_diagnostic_verbs = 100

kmc_verbs = KMeans(n_clusters = n_diagnostic_verbs)
verb_cluster_predictions = kmc_verbs.fit_predict(glove_vecs)

verb_cluster_pairs = list(enumerate(verb_cluster_predictions))

In [ ]:
def cluster_sample(l, n_clusters):
    res = []
    random.shuffle(l)
    for i in range(0, n_clusters):
        for v,c in l:
            if i == c:
                bv = v
        res += [bv]
    return(res)
        
import pandas as pd

def maketable():
    diagnostic_verb_list = [glove_verbs[i] for i in cluster_sample(verb_cluster_pairs, n_diagnostic_verbs)]
    diagnostic_frame_list = [frame_list[i] for i in cluster_sample(frame_cluster_pairs, n_diagnostic_frames)]

    if len(diagnostic_verb_list) - len(list(set(diagnostic_verb_list))) > 0:
        return(maketable())
    elif len(diagnostic_frame_list) - len(list(set(diagnostic_frame_list))) > 0:
        return(maketable())
    else:
        data = np.zeros((n_diagnostic_verbs, n_diagnostic_frames))
        for i in range(0, n_diagnostic_verbs):
            for j in range(0, n_diagnostic_frames):
                data[i][j] = verbnet_dict[diagnostic_verb_list[i]][ frame_list.index(diagnostic_frame_list[j]) ]

        df = pd.DataFrame(data, index = diagnostic_verb_list, columns = diagnostic_frame_list, dtype=int)


        for i in range(0, n_diagnostic_verbs//2):
            verb = pd.Series.argmin(df.sum(axis=1))
            df = df.drop(verb, axis=0)

        for i in range(0, n_diagnostic_frames//2):
            frame = pd.Series.argmin(df.sum(axis=0))
            df = df.drop(frame, axis=1)

        return(df)

btable = maketable()
bcount = pd.DataFrame.sum(df.sum(axis=0))
for i in range(0, 100):
    t = maketable()
    count = pd.DataFrame.sum(t.sum(axis=0))
    if count > bcount:
        btable = t
        bcount = count
df = btable

print("Sparsity = ", pd.DataFrame.sum(df.sum(axis=0)) / (n_diagnostic_frames//2 * n_diagnostic_verbs//2))
print("Size =", df.shape)